# Import Packages and Mount Google Drive

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.stats as sp
import pywt
from scipy import interpolate

from tensorflow import keras
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

.

.

.

.

# Load Raw dataset (1800 files)

- Tip Degradation Dataset (Spot_1 ~ 1800):
  - Robotic Spot-Welding (RSW)'s run-to-failure dataset collected from the initial state of the welding tip and continued through 1800 spot welds until the tip was completely worn out.

In [ ]:
# Define the size of dataset, number of sensor and features
NoOfData    = 1800  # 180 Data for each robotic spot-welding condition (Normal, Abnormal)
NoOfSensor  = 3    # 3 Sensor signals: Acceleration, Voltage, Current
NoOfFeature = 10   # 10 Feature types: Max, Min, Mean, RMS, Variance, Skewness, Kurtosis, Crest factor, Shape factor, Impulse factor

In [ ]:
#########################################################################################
# You can skip this cell and move onto the next cell to load pre-extracted feature data #
#########################################################################################

# Load all data from github
for i in range(NoOfData):
    
    temp_path = 'https://github.com/Eunseob/purdue_me597/blob/main/ml_tutorial/Dataset_tipWear/TipDegradationData(0~1800)/spot_%d?raw=true'%(i+1)
    exec("Spot_%d = pd.read_csv(temp_path , skiprows=1, names=['time(s)', 'acceleration(g)', 'voltage(V)', 'current(kA)'])"%(i+1))

# Define RMS function
def rms(x):
    return np.sqrt(np.mean(x**2))

# Set wavelet decomposition parameters
MotherWavelet = pywt.Wavelet('haar')
Level   = 8

# Build empty arrays for time and frqency featureset
TimeFeature = np.zeros(shape=(NoOfData, NoOfSensor*NoOfFeature))
FreqFeature = np.zeros(shape=(NoOfData, NoOfSensor*NoOfFeature*Level))

for i in range(NoOfData):
    
    # Declare temporary data
    exec("temp_data = Spot_%d"%(i+1))

    for j in range(NoOfSensor):

        # Extract time domain features
        TimeFeature[i, 10*j+0] = np.max(temp_data.iloc[:,j])
        TimeFeature[i, 10*j+1] = np.min(temp_data.iloc[:,j])
        TimeFeature[i, 10*j+2] = np.mean(temp_data.iloc[:,j])
        TimeFeature[i, 10*j+3] = rms(temp_data.iloc[:,j])
        TimeFeature[i, 10*j+4] = np.var(temp_data.iloc[:,j])
        TimeFeature[i, 10*j+5] = sp.skew(temp_data.iloc[:,j])
        TimeFeature[i, 10*j+6] = sp.kurtosis(temp_data.iloc[:,j])
        TimeFeature[i, 10*j+7] = np.max(temp_data.iloc[:,j])/rms(temp_data.iloc[:,j])
        TimeFeature[i, 10*j+8] = rms(temp_data.iloc[:,j])/np.mean(np.abs(temp_data.iloc[:,j]))
        TimeFeature[i, 10*j+9] = np.max(temp_data.iloc[:,j])/np.mean(np.abs(temp_data.iloc[:,j]))
        
        # Extract frequency domain features(WD)
        Coef = pywt.wavedec(temp_data, MotherWavelet, level=Level, axis=0)
        for k in range(Level):
            coef = Coef[Level-k]
            
            FreqFeature[i, NoOfFeature*j*Level+k*NoOfFeature+0] = np.max(coef[:,j])
            FreqFeature[i, NoOfFeature*j*Level+k*NoOfFeature+1] = np.min(coef[:,j])
            FreqFeature[i, NoOfFeature*j*Level+k*NoOfFeature+2] = np.mean(coef[:,j])
            FreqFeature[i, NoOfFeature*j*Level+k*NoOfFeature+3] = rms(coef[:,j])
            FreqFeature[i, NoOfFeature*j*Level+k*NoOfFeature+4] = np.var(coef[:,j])
            FreqFeature[i, NoOfFeature*j*Level+k*NoOfFeature+5] = sp.skew(coef[:,j])
            FreqFeature[i, NoOfFeature*j*Level+k*NoOfFeature+6] = sp.kurtosis(coef[:,j])
            FreqFeature[i, NoOfFeature*j*Level+k*NoOfFeature+7] = np.max(coef[:,j])/rms(coef[:,j])
            FreqFeature[i, NoOfFeature*j*Level+k*NoOfFeature+8] = rms(coef[:,j])/np.mean(np.abs(coef[:,j]))
            FreqFeature[i, NoOfFeature*j*Level+k*NoOfFeature+9] = np.max(coef[:,j])/np.mean(np.abs(coef[:,j]))

# Define names of sensor and feature
SensorList  = ['Acc', 'Vol', 'Cur']
FeatureList = ['Max', 'Min', 'Mean', 'RMS', 'Var', 'Skew', 'Kurt', 'Crest', 'Shape', 'Impulse']

TF_header,FF_header = [],[]

for Ss in SensorList:
    for Ft in FeatureList:
        TF_header.append('{}_{}'.format(Ss, Ft))
for Ss in SensorList:
    for dnum in range(Level):
        for Ft in FeatureList:
            FF_header.append('{}_D{}_{}'.format(Ss, dnum+1, Ft))

# Return the final feature dataset
TimeFeature = pd.DataFrame(TimeFeature, columns=TF_header)
FreqFeature = pd.DataFrame(FreqFeature, columns=FF_header)
FeatureData = pd.concat([TimeFeature, FreqFeature], axis = 1)
FeatureData.to_csv('/content/drive/MyDrive/Colab Notebooks/SavedFiles/ML10/FeatureData.csv', index=None)
FeatureData

In [ ]:
###########################################
# Load pre-extracted feature data(X_data) #
###########################################

FeatureData = pd.read_csv('https://github.com/Eunseob/purdue_me597/blob/main/ml_tutorial/Dataset_tipWear/FeatureData.csv?raw=true')
FeatureData

In [ ]:
########################################
# Load tip area data for label(Y_data) #
########################################

TipArea_label = pd.read_csv('https://github.com/Eunseob/purdue_me597/blob/main/ml_tutorial/Dataset_tipWear/TipArea_Label?raw=true')
TipArea_label

.

.

.

# Feature Selection
- Unlike the most distinguishable features were selected for classification, the features having highest correlation with domain or label should be selected for real-value prediction.
- Pearson Correlation Coefficient (PCC) is generally used for this type of feature selection. 

In [ ]:
NoOfFeature_total = FeatureData.shape[1]
FeatureCorr       = pd.DataFrame(np.zeros(shape=(NoOfFeature_total, 2)))   # Empty array for PCC evaluation result

for i in range(NoOfFeature_total):
    tempX = pd.Series(range(1, FeatureData.shape[0]+1)) # Number of welding spots (1~1800)
    tempY = pd.Series(FeatureData.iloc[:,i].values)     # Each type of feature (1~270)

    PrsCorr = np.abs(pd.concat( [tempX, tempY] , axis=1).corr(method='pearson').iloc[0,1]) # Absolute value of PCC
    
    FeatureCorr.iloc[i,0] = FeatureData.iloc[:,i].name  # Name of the feature
    FeatureCorr.iloc[i,1] = PrsCorr                     # Absolute value of PCC

FeatureCorr.to_csv('/content/drive/MyDrive/Colab Notebooks/SavedFiles/ML10/FeatureCorr.csv', index=None)
FeatureCorr_sorted = FeatureCorr.sort_values(by=[1], ascending = False)
FeatureCorr_sorted.T

In [ ]:
# Select top 30 features having highest PCC
Rank = 30

FeatureCorr_Selected = FeatureCorr_sorted.iloc[:Rank, 0]
FeatureCorr_Selected = list(FeatureCorr_Selected)
FeatureSelected_1800 = FeatureData[FeatureCorr_Selected]

# Select the final features from 60th spot that have label
FeatureSelected = FeatureSelected_1800.iloc[60:,:]
FeatureSelected.to_csv('/content/drive/MyDrive/Colab Notebooks/SavedFiles/ML10/FeatureSelected.csv', index=None)
FeatureSelected

.

.

.

.

.

# Data and Label Preparation for RNN

In [ ]:
# Data Scaling (manually)
Statistics = np.zeros((4, FeatureSelected.shape[1]))
for i in range(FeatureSelected.shape[1]):
    FeatureNow = FeatureSelected.iloc[:,i].values
    Statistics[:,i] = np.min(FeatureNow), np.max(FeatureNow), np.mean(FeatureNow), np.std(FeatureNow)
    
# Normalization (MinMax Scalering)
FeatureSelected_scaled = (FeatureSelected - Statistics[0,:]) / (Statistics[1,:] - Statistics[0,:])

# Standardization (Standard Scalering)
# FeatureSelected_scaled = (FeatureSelected - Statistics[2,:]) / Statistics[3,:]

FeatureSelected_scaled.shape

In [ ]:
# Label Scaling (manually)
TipArea_label_scaled = TipArea_label.iloc[:,1] / np.max(TipArea_label.iloc[:,1])
TipArea_label_scaled.shape

In [ ]:
seq_length = 10 # Use 10 spots of data for a sequence

Data, Label = [], []
for t in range(len(TipArea_label_scaled)-seq_length):
    temp_X = FeatureSelected_scaled[t : t+seq_length]  #  Data(t): Feature(t-10) ~ Feature(t)
    temp_Y = TipArea_label_scaled[t+seq_length]        # Label(t): TipArea(t)    
    Data.append(temp_X)
    Label.append(temp_Y)

Data  = np.array(Data)
Label = np.array(Label).reshape(len(Label), 1)

Data.shape, Label.shape

In [ ]:
# Test data/label: every 5th data/label
# Training data/label: remaining data/label

TrainData = np.concatenate([Data[1::5], Data[2::5], Data[3::5], Data[4::5]], axis = 0) # 1,2,3,4,  6,7,8,9,   11,12,13,14,   ....
TestData  = Data[::5]                                                                  #         5,        10,            15,....

TrainLabel = np.concatenate([Label[1::5], Label[2::5], Label[3::5], Label[4::5]], axis = 0) # 1,2,3,4,  6,7,8,9,   11,12,13,14,   ....
TestLabel = Label[::5]                                                                      #         5,        10,            15,....
 
TrainData.shape, TestData.shape, TrainLabel.shape, TestLabel.shape

.

.

.

.

.

# RNN Model Training

- Types of Recurrent layer: https://keras.io/api/layers/recurrent_layers/

In [ ]:
def RNN_model(inputData):
    keras.backend.clear_session() # Clear session

    model = keras.Sequential()
    
    model.add(keras.layers.InputLayer(input_shape=[inputData.shape[1], inputData.shape[2]]))
    model.add(keras.layers.SimpleRNN(units = 32, name='RNN_cell_1', return_sequences=True)) # Activation: 'tanh' by default
    model.add(keras.layers.SimpleRNN(units = 32, name='RNN_cell_2'))
    model.add(keras.layers.Dense(1, name='output'))
    
    model._name = 'stacked_RNN'
    model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.001),
                  loss = 'mse', metrics=['mse', 'mae'])
    
    return model

In [ ]:
RnnModel = RNN_model(TrainData)
RnnModel.summary()

In [ ]:
tf.random.set_seed(777)
TraingHistory = RnnModel.fit(TrainData, TrainLabel, validation_data=(TestData, TestLabel), epochs = 100, verbose = 1)

In [ ]:
# Confirm the training history
History = pd.DataFrame(TraingHistory.history)

plt.figure(figsize=(6,4))
plt.plot(History[['mse']]    , label='training loss')
plt.plot(History[['val_mse']], label='validation loss')

plt.title('Training History for RNN')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# RNN model save, (re)load, and predict
RnnModel.save('/content/drive/MyDrive/Colab Notebooks/SavedFiles/ML_Models/RNN_model.h5')
LoadedModel = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/SavedFiles/ML_Models/RNN_model.h5')

Predicted = LoadedModel.predict(TestData)               # Predicted result: Scaled 
Predicted = Predicted * np.max(TipArea_label.iloc[:,1]) # Inverse Scaling

In [ ]:
# Plot 'Predicted vs. Actual Tip Area of RSW'
SpotNumber = np.arange(70, 1800, 5)
Actual = TipArea_label.iloc[seq_length:,1][::5]         # Every 5th tip Area from 70th spot

plt.figure(figsize = (10,6))

plt.plot(SpotNumber, Predicted, c = (1.0, 0.2, 0.2, 0.8), label = 'Predicted Tip Area', linestyle='', marker='o')
plt.plot(SpotNumber, Actual   , c = (0.2, 0.2, 1.0, 0.8), label = 'Actual Tip Area')

plt.title('Predicted vs. Actual Tip Area of RSW', fontsize=15)
plt.xlabel('Spot')
plt.ylabel('Tip Area ($mm^2$)')
plt.legend()
plt.grid()
plt.show()